In [1]:
import pandas as pd

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv('./solicitacoescredito.csv')

# Conta os valores nulos em cada coluna
contagem_nulos = df.isnull().sum()

# Exibe a contagem de valores nulos
print(contagem_nulos)


numero_solicitacao                0
razaoSocial                       0
nomeFantasia                      0
cnpjSemTraco                      0
maiorAtraso                       0
margemBrutaAcumulada              0
percentualProtestos            1498
primeiraCompra                  106
prazoMedioRecebimentoVendas       0
titulosEmAberto                   0
valorSolicitado                   0
status                            0
definicaoRisco                    0
diferencaPercentualRisco          0
percentualRisco                   0
dashboardCorrelacao               0
valorAprovado                  1404
dataAprovadoEmComite           8415
periodoBalanco                 4240
ativoCirculante                4240
passivoCirculante              4240
totalAtivo                     4240
totalPatrimonioLiquido         4240
endividamento                  4240
duplicatasAReceber             4240
estoque                        4240
faturamentoBruto                750
margemBruta                 

In [5]:
import pandas as pd

# Seleciona apenas as colunas numéricas
colunas_numericas = df.select_dtypes(include=['number'])

# Calcula a média de cada coluna numérica
medias = colunas_numericas.mean()

# Preenche os valores nulos com as médias
df_preenchido = df.fillna(medias)

# Salva o DataFrame preenchido de volta para um arquivo CSV
df_preenchido.to_csv('solicitacoesFiltradas.csv', index=False)


In [6]:
import pandas as pd

# Carrega o arquivo CSV para um DataFrame
df = pd.read_csv('./solicitacoesFiltradas.csv')

# Conta os valores nulos em cada coluna
contagem_nulos = df.isnull().sum()

# Exibe a contagem de valores nulos
print(contagem_nulos)

numero_solicitacao                0
razaoSocial                       0
nomeFantasia                      0
cnpjSemTraco                      0
maiorAtraso                       0
margemBrutaAcumulada              0
percentualProtestos               0
primeiraCompra                  106
prazoMedioRecebimentoVendas       0
titulosEmAberto                   0
valorSolicitado                   0
status                            0
definicaoRisco                    0
diferencaPercentualRisco          0
percentualRisco                   0
dashboardCorrelacao               0
valorAprovado                     0
dataAprovadoEmComite           8415
periodoBalanco                 4240
ativoCirculante                   0
passivoCirculante                 0
totalAtivo                        0
totalPatrimonioLiquido            0
endividamento                     0
duplicatasAReceber                0
estoque                           0
faturamentoBruto                  0
margemBruta                 

In [9]:
import pandas as pd



# Lista das colunas de interesse
colunas_interesse = ['percentualProtestos', 'status', 'definicaoRisco', 'valorAprovado', 
                     'endividamento', 'passivoCirculante', 'totalAtivo', 'totalPatrimonioLiquido', 
                     'faturamentoBruto', 'scorePontualidade', 'percentualRisco']

# Verifica se há valores nulos nessas colunas
valores_nulos = df[colunas_interesse].isnull().any()

# Exibe os resultados
print(valores_nulos)

percentualProtestos       False
status                    False
definicaoRisco            False
valorAprovado             False
endividamento             False
passivoCirculante         False
totalAtivo                False
totalPatrimonioLiquido    False
faturamentoBruto          False
scorePontualidade         False
percentualRisco           False
dtype: bool


In [10]:
dados_interesse = df[colunas_interesse]

# Critérios de seleção para clientes ruins
crit_ruins = (dados_interesse['percentualProtestos'] > 0) | \
             (dados_interesse['definicaoRisco'] != 'De 0 a 10 % - Muito Baixo') | \
             (dados_interesse['valorAprovado'] < 50000) | \
             (dados_interesse['endividamento'] != 0) | \
             (dados_interesse['totalPatrimonioLiquido'] < 10000) | \
             (dados_interesse['faturamentoBruto'] < 500000)

# Critérios de seleção para clientes bons
crit_bons = (dados_interesse['percentualProtestos'] == 0) & \
            (dados_interesse['definicaoRisco'] == 'De 0 a 10 % - Muito Baixo') & \
            (dados_interesse['valorAprovado'] >= 50000) & \
            (dados_interesse['endividamento'] == 0) & \
            (dados_interesse['totalPatrimonioLiquido'] >= 10000) & \
            (dados_interesse['faturamentoBruto'] >= 500000)


# Filtrando o DataFrame original com base nos critérios
clientes_ruins = df[crit_ruins]
clientes_bons = df[crit_bons]

# Salvando os resultados em arquivos CSV
clientes_ruins.to_csv('clientes_ruins.csv', index=False)
clientes_bons.to_csv('clientes_bons.csv', index=False)


In [11]:
# Conta o número de clientes bons e ruins
num_clientes_bons = len(clientes_bons)
num_clientes_ruins = len(clientes_ruins)

# Exibe o resultado
print("Número de clientes bons:", num_clientes_bons)
print("Número de clientes ruins:", num_clientes_ruins)

Número de clientes bons: 315
Número de clientes ruins: 8658


In [15]:
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Carrega os arquivos CSV dos clientes bons e ruins
clientes_bons = pd.read_csv('clientes_bons.csv')
clientes_ruins = pd.read_csv('clientes_ruins.csv')

# Adiciona uma coluna 'target' para identificar clientes bons (1) e ruins (0)
clientes_bons['target'] = 1
clientes_ruins['target'] = 0

# Combina os DataFrames de clientes bons e ruins
df = pd.concat([clientes_bons, clientes_ruins])

# Convertendo variáveis categóricas em variáveis dummy
df = pd.get_dummies(df)

# Define os recursos (X) e os rótulos (y)
X = df.drop(columns=['target'])
y = df['target']


# Aplica o SMOTE para balancear as classes
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X, y)

# Divide os dados em conjunto de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

# Inicializa e treina o modelo RandomForestClassifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

# Avalia o modelo
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1741
           1       1.00      1.00      1.00      1723

    accuracy                           1.00      3464
   macro avg       1.00      1.00      1.00      3464
weighted avg       1.00      1.00      1.00      3464

